# Walkthrough Challenge 3 - Chat with structured data using Plugins and Planners

Duration: 45 minutes

## Overview
- In this challenge, you will learn how to create a Plugin and invoke it from the kernel.
- You will also learn how to use a Planner to execute an automated sequence of steps, ultimately creating a chatbot that can interact with structured data.

## Prerequisites

- Please ensure that you have completed the [Setup](../setup/setup.ipynb) before starting this challenge.

### Task 1: Configure and Initialize Semantic Kernel

⚠️ Note: You should have already completed all tasks on the [Setup](../setup/setup.ipynb). If you have not, please go back and complete it now.

#### Step 1: Load Semantic Kernel settings

In this step, we will load the Semantic Kernel settings that we created in the [Setup](../setup/setup.ipynb) notebook.

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.13.0"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.13.0-preview"
#r "nuget: Microsoft.SemanticKernel.Planners.OpenAI, 1.13.0-preview"
#r "nuget: System.Data.SQLite, 1.0.118"
#r "nuget: Dapper, 2.1.24"
#r "nuget:Microsoft.ML.DataView"
#r "nuget:Microsoft.Data.Analysis"

#!import ../setup/config/Settings.cs
#!import ../setup/config/Utils.cs
#!import plugins/SqlPlugin/SqlPlugin.cs

#### Step 2: Initialize Semantic Kernel

In [ ]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.TemplateEngine;
using Kernel = Microsoft.SemanticKernel.Kernel;
using Microsoft.DotNet.Interactive;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;
using Microsoft.Data.Analysis;
using Microsoft.ML;

var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile("../setup/config/settings.json");

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

### Task 2: Create and initialize the `SqlPlugin` class

Let's introduce you to Semantic Kernel Plugins, which one of the fundamental building blocks of Semantic Kernel. Plugins are used to encapsulate capabalities into a single unit of functionality that can be run by the kernel. Plugins can either consist of native code or requests to an LLM using a prompt.

Take a look at the following Semantic Kernel documentantion about Plugins: [What is a plugin?](https://learn.microsoft.com/en-us/semantic-kernel/agents/plugins/?tabs=Csharp#what-is-a-plugin)

In this example, we are going to create a new Semantic Kernel plugin that will allow us to execute SQL queries against a SQLite database. We will use the Dapper library to execute the queries and return the results as a table.

You can find the plugin code in the [`SqlPlugin.cs`](./plugins/SqlPlugin/SqlPlugin.cs) file. This plugin is a simple example of how to create a plugin that can execute SQL queries against an actual relational database.

Note that it has three functions: `GetTables`, `GetSchema` and `RunQuery`.

- `GetTables` function returns a list of tables in the database.
- `GetSchema` function returns the schema of a table.
- `RunQuery` function executes a SQL query and returns the results as a csv string.

Each function is decorated with the `KernelFunction` attribute, which tells the kernel that this method is a native function and will automatically register it with the kernel when the plugin is loaded.

Notice how we've added a description to the function and each of its parameters with the `Description` attribute. This description will be used by function calling and by planners to automatically create a plan using these functions. More on this later.

### Task 3: Load the `SqlPlugin` and invoke its functions

We have seen how to create a plugin, now let's see how to load it into the kernel. We will use the `ImportPluginFromObject` method to load the plugin into the kernel.

In [ ]:
kernel.ImportPluginFromObject(new SqlPlugin());

Let's invoke a function from the plugin to see how it works. We will use the `GetTables` function to get a list of tables in the database.

It will make an actual query to the [northwind.db](./northwind.db) database, which is a SQLite database that contains the Northwind sample database.

In [ ]:
var getTablesFunction = kernel.Plugins["SqlPlugin"]["GetTables"];

var getTablesResult = await kernel.InvokeAsync(getTablesFunction, new() {});
DataFrame tables = DataFrame.LoadCsvFromString(getTablesResult.ToString());

tables.Display();

As you can see, the `GetTables` function was able to return a list of tables in the database. This is a simple example of how to create a plugin and load it into the kernel.

Let's invoke the `GetSchema` function, which will return the schema of the `Orders` table.

In [ ]:
var getSchemaFunction = kernel.Plugins["SqlPlugin"]["GetSchema"];

var getSchemaResult = await kernel.InvokeAsync(getSchemaFunction, new() { { "tableName", "Orders" } });
DataFrame categoriesColumns = DataFrame.LoadCsvFromString(getSchemaResult.ToString());

categoriesColumns.Display();

### Task 4: Create a plan to execute a SQL query

Now that we have a plugin that can execute SQL queries, let's see how we can create a plan to execute a SQL query based on a natural language question.
So far, we have seen how to create a plugin and load it into the kernel. We have also seen how to invoke a function from the plugin to get a list of tables in the database.

We will use a planner that will take a natural language question and create a plan to execute a SQL query based on that question. you can see more about Semantic Kernel Planners in here: [Automatically orchestrate AI with planners](https://learn.microsoft.com/en-us/semantic-kernel/agents/planners/?tabs=Csharp).

Behind the scenes, planner uses an LLM prompt to generate a plan. The plan is then executed by the kernel, which will call the appropriate plugin function to achieve the desired response.

In [ ]:
using Microsoft.SemanticKernel.Planning.Handlebars;
using Microsoft.SemanticKernel.Planning;

#pragma warning disable SKEXP0061
#pragma warning disable SKEXP0060


var config = new FunctionCallingStepwisePlannerOptions
{
    MaxIterations = 15,
    MaxTokens = 8000,
};
var planner = new FunctionCallingStepwisePlanner();


We are going to use the `FunctionCallingStepwisePlanner` class to create a plan to execute a SQL query based on a natural language question.

The planner will use the `SqlPlugin` and its functions to create a plan to execute a SQL query based on a natural language question.

In [ ]:
#pragma warning disable SKEXP0061
#pragma warning disable SKEXP0060

var ask = "What are the top 10 most sold products?";
var result = await planner.ExecuteAsync(kernel, ask);

Console.WriteLine("\nResult:");
Console.WriteLine(result.FinalAnswer);

You successfully completed challenge 3! 🚀🚀🚀

 **[Home](../../Readme.md)** - [Next Challenge Solution](../challenge-4/solution.ipynb)